skip-gram 以及 負採樣
- (target_word, context_word): context_word就是target_word周圍的字詞，使用target_word去預測context_word。
- ![alt target_word, context_word解說](https://tensorflow.org/tutorials/text/images/word2vec_skipgram.png?hl=zh-cn)
- 其中使用了NCE loss去近似full softmax，藉由此loss function不用去建構整個word的預測分配，而是變成負樣本與正樣本之間的分類問題。

In [16]:
import io
import re
import string
import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

建立樣本samples

In [19]:
sentence = 'The wide road shimmered in the hot sun'
tokens = list(sentence.lower().split())
print(len(tokens), tokens)

8 ['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


建立一個vocabulary 去從token轉換到index

In [30]:
vocab, index = {}, 1
vocab['<pad>'] = 0
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1

vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


建立一個inversed vocabulary 去將index轉換回token

In [31]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


向量化sentence

In [32]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


產生skip-grams的樣本從一句話中
- [skipgram](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/skipgrams)

In [33]:
# 正樣本

window_size = 2
positive_skip_grams, _ = keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0      # 因為負採樣比較複雜，額外處理。這裡代表不產生負樣本
)
print(len(positive_skip_grams))
print(positive_skip_grams)

26
[[1, 2], [2, 3], [4, 5], [3, 1], [6, 1], [2, 4], [2, 1], [1, 3], [5, 4], [7, 1], [6, 5], [1, 5], [1, 4], [3, 2], [3, 5], [1, 7], [4, 3], [5, 6], [7, 6], [5, 1], [4, 1], [6, 7], [3, 4], [5, 3], [1, 6], [4, 2]]


轉換回token看一些筆數

In [34]:
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(1, 2): (the, wide)
(2, 3): (wide, road)
(4, 5): (shimmered, in)
(3, 1): (road, the)
(6, 1): (hot, the)


產出負樣本: 隨機抽取words從vocabulary，可以使用tf.random.log_uniform_candidate_sampler function去抽取num_ns的負樣本給予給定的target_word 在一個window之中。使用這個function並且跟正樣本輸入為context word來排除這些。

In [37]:
# 負樣本抽取

target_word, context_word = positive_skip_grams[0]
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype='int64'), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class, # 正樣本需要被排除的
    num_true=1,         # 每一個正樣本有一個正確的class
    num_sampled=num_ns,
    unique=True,        # 負樣本不要重複
    range_max=vocab_size,   # (0, vocab_size)
    seed=SEED,
    name="negative_sampling"
)
print(sentence)
print(inverse_vocab[target_word])
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

The wide road shimmered in the hot sun
the
tf.Tensor([1 5 0 4], shape=(4,), dtype=int64)
['the', 'in', '<pad>', 'shimmered']


建構訓練樣本，將剛剛的正樣本以及負樣本合起來。

In [38]:
# 新增一個維度方便之後的concatenation
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, axis=1)

# 合併正樣本以及負樣本, (1+num_ns, 1)
context = tf.concat([context_class, negative_sampling_candidates], axis=0)

# label
label = tf.constant([1] + [0]*num_ns, dtype='int64')

# reshape
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [39]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 1
target_word     : the
context_indices : [2 1 5 0 4]
context_words   : ['wide', 'the', 'in', '<pad>', 'shimmered']
label           : [1 0 0 0 0]


一個tuple的(target, context, label) 的tensors 組成了一個訓練樣本for skip-gram 負採樣模型。(1+num_ns, )

In [40]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(1, shape=(), dtype=int32)
context : tf.Tensor([2 1 5 0 4], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


下圖為目前處理的結構圖
![](https://tensorflow.org/tutorials/text/images/word2vec_negative_sampling.png?hl=zh-cn)

將上述模組化

In [41]:
# 因為一些常用字通常不會帶來訊息，可以用一個學者建議的方法，將常用字進行二次抽樣來提高資料質量

sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


sampling_table[i] 表示對數據集中第 i 個最常見單詞進行採樣的概率。該函數採用 Zipf 分佈的詞頻進行採樣。

In [69]:
# 將上述的操作function化

def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    targets, contexts, labels = [], [], []

    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    for sequence in tqdm.tqdm(sequences):
        # 產生正樣本
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0
        )

        # 產生負樣本
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                tf.constant([context_word], dtype='int64'),
                1
            )
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=SEED,
                name='negative_sampling'
            )
            # 向量化
            negative_sampling_candidates = tf.expand_dims(
                negative_sampling_candidates, 1
            )
            context = tf.concat([context_class, negative_sampling_candidates], axis=0)
            label = tf.constant([1] + [0] * num_ns, dtype='int64')

            # append
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)
    return targets, contexts, labels

準備資料，使用內建的資料集。

In [70]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [71]:
with open(path_to_file) as f:
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


使用非空行判斷式構造一個 tf.data.TextLineDataset 對像以進行後續步驟：

In [72]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

向量化句子 from the corpus

In [73]:
def custom_standization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation), '')


vocab_size = 4096
sequence_length = 10

# 使用TextVectorization layer去正規畫、split、map strings to integers。

vectorize_layer = layers.TextVectorization(
    standardize=custom_standization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

呼叫.adapt去創造vocabulary

In [74]:
vectorize_layer.adapt(text_ds.batch(1024))

一旦層的狀態被調整為表示文本語料庫，就可以使用 TextVectorization.get_vocabulary 訪問詞彙表。此函數返回按頻率排序（降序）的所有詞彙標記的列表。

In [75]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [76]:

# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

取得seqs from the dataset

In [77]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [78]:
# 看一些

for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


產生訓練樣本

In [79]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED
)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:47<00:00, 694.42it/s]




targets.shape: (65271,)
contexts.shape: (65271, 5)
labels.shape: (65271, 5)


In [81]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)


dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>
<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [82]:
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = layers.Embedding(vocab_size,
                                    embedding_dim,
                                    input_length=1,
                                    name="w2v_embedding")
        self.context_embedding = layers.Embedding(vocab_size,
                                    embedding_dim,
                                    input_length=num_ns+1)

    def call(self, pair):
        target, context = pair
        # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
        # context: (batch, context)
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)
        # target: (batch,)
        word_emb = self.target_embedding(target)
        # word_emb: (batch, embed)
        context_emb = self.context_embedding(context)
        # context_emb: (batch, context, embed)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        # dots: (batch, context)
        return dots

In [83]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [88]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 #loss=custom_loss,
                 metrics=['accuracy'])

In [89]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [90]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 3s 6ms/step - loss: 1.6082 - accuracy: 0.2319
Epoch 2/20
63/63 [==============================] - 0s 4ms/step - loss: 1.5888 - accuracy: 0.5563
Epoch 3/20
63/63 [==============================] - 0s 3ms/step - loss: 1.5412 - accuracy: 0.5988
Epoch 4/20
63/63 [==============================] - 0s 3ms/step - loss: 1.4588 - accuracy: 0.5744
Epoch 5/20
63/63 [==============================] - 0s 3ms/step - loss: 1.3607 - accuracy: 0.5810
Epoch 6/20
63/63 [==============================] - 0s 3ms/step - loss: 1.2636 - accuracy: 0.6072
Epoch 7/20
63/63 [==============================] - 0s 3ms/step - loss: 1.1731 - accuracy: 0.6420
Epoch 8/20
63/63 [==============================] - 0s 3ms/step - loss: 1.0891 - accuracy: 0.6746
Epoch 9/20
63/63 [==============================] - 0s 3ms/step - loss: 1.0112 - accuracy: 0.7062
Epoch 10/20
63/63 [==============================] - 0s 3ms/step - loss: 0.9387 - accuracy: 0.7376
Epoch 11/20
63/63 [

In [95]:
for x, y in dataset:
    print(x)
    print(word2vec(x))
    break

(<tf.Tensor: shape=(1024,), dtype=int64, numpy=array([ 145,  580, 1447, ..., 2930,  624, 2538])>, <tf.Tensor: shape=(1024, 5), dtype=int64, numpy=
array([[   6,    0,   60, 1675,  590],
       [ 320,  359,   12,  605,    0],
       [ 395,  403,    9,  136,  569],
       ...,
       [  19,  356,   72,   44,   58],
       [  30,   67,   30,    3,    7],
       [   1,    4,   53,   11,  847]])>)
tf.Tensor(
[[ 0.16194773 -3.1330385  -0.5472963  -1.1153789  -0.8942378 ]
 [ 3.7518773  -1.0048577   1.4820281   0.33710948 -2.5655465 ]
 [ 1.5666049  -0.89673465 -0.5126023  -1.0258136  -0.6283038 ]
 ...
 [ 3.9270492  -0.21464214  0.21112093 -0.8593108  -0.07414787]
 [ 3.3815556   1.429946    3.3815556   1.2083132   0.7439871 ]
 [ 2.3059983  -0.4030246  -0.71685183  0.28747374  0.01626177]], shape=(1024, 5), dtype=float32)
